# Depth Image Transformations

In [1]:
import numpy as np
import math
import cv2
import os, os.path
import sys
import time
import pandas as pd

path = 'C:/Users/jumpr_000/Desktop/ransac_obstacle_detection/saved' #set to your desired path to save images in
frames_dir = "test_frames_straight/" #the directory holding the saved depth images in .npy format

#camera information based on the Kinect v2 hardware
CameraParams = {
  "cx":254.878,
  "cy":205.395,
  "fx":365.456,
  "fy":365.456,
  "k1":0.0905474,
  "k2":-0.26819,
  "k3":0.0950862,
  "p1":0.0,
  "p2":0.0,
}

# Kinect's physical orientation in the real world.
CameraPosition = {
    "x": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "y": 0, # actual position in meters of kinect sensor relative to the viewport's center.
    "z": 0, # height in meters of actual kinect sensor from the floor.
    "roll": 0, # angle in degrees of sensor's roll (used for INU input - trig function for this is commented out by default).
    "azimuth": 0, # sensor's yaw angle in degrees.
    "elevation": -30, # sensor's pitch angle in degrees.
}

def depthMatrixToPointCloudPos(z, scale=1000):
    """
    xyz_arr = depthMatrixToPointCloudPos(depth_frame)

    Given a depth image, converts each
    point to an XYZ coordinate and returns
    an array of these points.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    C, R = np.indices(z.shape)

    R = np.subtract(R, CameraParams['cx'])
    R = np.multiply(R, z)
    R = np.divide(R, CameraParams['fx'] * scale)

    C = np.subtract(C, CameraParams['cy'])
    C = np.multiply(C, z)
    C = np.divide(C, CameraParams['fy'] * scale)

    return np.column_stack((z.ravel() / scale, R.ravel(), -C.ravel()))

def depthToPointCloudPos(x_d, y_d, z, scale=1000):
    """
    x, y, z = (row, col, depth)
    
    Given a point from a depth image
    and its position in the image,
    returns the x, y, and z coordinates
    relative to the camera location.
    The scale is a conversion factor.
    Default converts from milimeter
    depth data to meters.
    """
    x = (x_d - CameraParams['cx']) * z / CameraParams['fx']
    y = (y_d - CameraParams['cy']) * z / CameraParams['fy']

    return x / scale, y / scale, z / scale

def applyCameraOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # This runs slowly in Python as it is required to be called within a loop, but it is a more intuitive example than it's vertorized alternative (Purly for example)
    # use trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[ax1] ** 2 + pt[ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[ax2], pt[ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(0, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(1, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y plane

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

def applyCameraMatrixOrientation(pt):
    # Kinect Sensor Orientation Compensation
    # bacically this is a vectorized version of applyCameraOrientation()
    # uses same trig to rotate a vertex around a gimbal.
    def rotatePoints(ax1, ax2, deg):
        # math to rotate vertexes around a center point on a plane.
        hyp = np.sqrt(pt[:, ax1] ** 2 + pt[:, ax2] ** 2) # Get the length of the hypotenuse of the real-world coordinate from center of rotation, this is the radius!
        d_tan = np.arctan2(pt[:, ax2], pt[:, ax1]) # Calculate the vertexes current angle (returns radians that go from -180 to 180)

        cur_angle = np.degrees(d_tan) % 360 # Convert radians to degrees and use modulo to adjust range from 0 to 360.
        new_angle = np.radians((cur_angle + deg) % 360) # The new angle (in radians) of the vertexes after being rotated by the value of deg.

        pt[:, ax1] = hyp * np.cos(new_angle) # Calculate the rotated coordinate for this axis.
        pt[:, ax2] = hyp * np.sin(new_angle) # Calculate the rotated coordinate for this axis.

    #rotatePoints(1, 2, CameraPosition['roll']) #rotate on the Y&Z plane # Disabled because most tripods don't roll. If an Inertial Nav Unit is available this could be used)
    rotatePoints(0, 2, CameraPosition['elevation']) #rotate on the X&Z plane
    rotatePoints(0, 1, CameraPosition['azimuth']) #rotate on the X&Y

    # Apply offsets for height and linear position of the sensor (from viewport's center)
    pt[:] += np.float_([CameraPosition['x'], CameraPosition['y'], CameraPosition['z']])
    return pt

# Plane Fitting

In [2]:
def plane_fit(points):
    """
    p, n = plane_fit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

def get_orientation(xyz_arr, n_iter):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, n_iter):
        rand_points = []
        while len(rand_points) < 10:
            index = random.randrange(0,len(xyz_arr))
            if not xyz_arr[index].all() == np.zeros(3).all():
                rand_points.append(xyz_arr[index])
        rand_points = np.array(rand_points).T
        ctr, P = plane_fit(rand_points)
        r = math.sqrt(P[0]**2 + P[1]**2 + P[2]**2)
        theta = math.acos(P[2]/r) * 180 / math.pi
        phi = math.atan(P[1]/P[0]) * 180 / math.pi
        pitch.append(theta)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    
    
from sklearn.linear_model import RANSACRegressor
from sklearn.datasets import make_regression

In [3]:
# Inital guess of the plane
p0 = [0.506645455682, -0.185724560275, -1.43998120646, 1.37626378129]

def f_min(X,p):
    plane_xyz = p[0:3]
    #print('plane_xyz', plane_xyz)
    distance = (plane_xyz*X).sum(axis=1) + p[3]
    return distance / np.linalg.norm(plane_xyz)

def residuals(params, signal, X):
    return f_min(X, params)

from scipy.optimize import leastsq

def fit_plane_optimize(arr):
    arr = np.random.permutation(arr)
    arr = np.reshape(arr, (np.shape(arr)[0], -1))
    ctr = arr.mean(axis=1)
    sol = leastsq(residuals, p0, args=(None, arr))[0]
    plane_xyz = sol[0:3]
    r = math.sqrt(plane_xyz[0]**2 + plane_xyz[1]**2 + plane_xyz[2]**2)
    theta = math.acos(plane_xyz[2]/r) * 180 / math.pi
    phi = math.atan(plane_xyz[1]/plane_xyz[0]) * 180 / math.pi
    return ctr[0:3], plane_xyz, phi

def get_orientation(xyz_arr, num_planes, num_points):
    """
    center, plane, theta = get_orientation(xyz_arr, num_points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fits a plane to the array n_iter times and
    returns the average of the planes centers,
    norms, and pitch degrees.
    """
    pitch = []
    planes = []
    centers = []
    for _ in range(0, num_planes):
        rand_points = np.random.permutation(xyz_arr[np.nonzero(xyz_arr)])[:num_points]
        #print('Random points:\n', rand_points)
        #print('Transpose:\n', rand_points.T)
        ctr, P, phi = fit_plane_optimize(rand_points.T)
        pitch.append(phi)
        planes.append(P)
        centers.append(ctr)
    return np.mean(centers, axis = 0), np.mean(planes, axis = 0), np.mean(pitch)
    

# Obstacle Detection

In [9]:
import numpy as np
import math
import random
import cv2
import os, os.path
import sys
import time
import pandas as pd
from pyntcloud import PyntCloud
from IPython.display import display, HTML

def get_obstacles_with_plane(depth_frame, num_planes, num_points, dist_thresh, visualize):
    obstacles = np.zeros(depth_frame.shape) #empty image that will store the locations of detected obstacles
    img = np.uint8(depth_frame) #some opencv functions require a byte image
    kernel = np.ones((3,3),np.uint8)
    _, foreground = cv2.threshold(depth_frame,0.75,1.75, cv2.THRESH_TOZERO) 
    denoised = cv2.morphologyEx(depth_frame, cv2.MORPH_OPEN, kernel)
    xyz_arr = depthMatrixToPointCloudPos(denoised) #convert depth data to XYZ coordinates
    #print(fit_plane_optimize(xyz_arr.T))
    start_time = time.time()
    center, plane, theta = get_orientation(xyz_arr.T, num_planes, num_points)
    print(center, plane, theta)
    #center, plane, theta = fit_plane_optimize(xyz_arr.T, num_points)
    CameraPosition['elevation'] = -theta
    center = applyCameraOrientation(center)
    plane = applyCameraOrientation(plane)
    xyz_arr = applyCameraMatrixOrientation(xyz_arr)
    plane_img = np.zeros(len(xyz_arr))
    plane_img[xyz_arr[:,2] > dist_thresh - center[2]] = 1
    if visualize:
        xyz_arr = xyz_arr[xyz_arr[:,2] > dist_thresh - center[2]]
        points = pd.DataFrame(xyz_arr, columns=['x', 'y', 'z']) #convert XYZ coordinates to a DataFrame for pyntcloud
        '''
        cloud = PyntCloud(points)

        lines = [
            {
                # X axis
                "color": "red", 
                "vertices": [[0, 0, 0], [10, 0, 0]]
            },
            {
                # Y axis
                "color": "green",
                "vertices": [[0, 0, 0], [0, 10, 0]]
            },
            {
                # Z axis
                "color": "blue",
                "vertices": [[0, 0, 0], [0, 0, 10]]
            },
            {
                #Original norm of the plane
                "color": "pink",
                "vertices": [center.tolist(), plane.tolist()]
            }
        ]
        cloud.plot(cmap="cool", polylines=lines)
        '''

    plane_img = np.uint8(np.reshape(plane_img,(424,512)) * 255) #reshape to match depth data and convert to uint8
    plane_img = np.uint8((np.ones((424,512)) * 255) - plane_img) #invert img so pixel value corresponds to NOT ground plane
    ret, plane_img = cv2.threshold(plane_img,0,255,cv2.THRESH_BINARY) #filter points that are probaly not ground plane
    plane_img = cv2.subtract(img, plane_img)
    
    #noise removal
    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(plane_img ,cv2.MORPH_OPEN, kernel, iterations = 3) #erosion followed by dilation
    erode = cv2.erode(opening, kernel, iterations=6)
    _, thresh = cv2.threshold(erode,0,255,cv2.THRESH_TOZERO)
    color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) #BGR image to draw labels on

    #begin contour detection
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    color = cv2.drawContours(color, contours, -1, (0,255,0), 1)
    for cntr in contours:
        try:
            #calculate diamter of equivalent circle
            #this measurement is only used for checking if countours fit our bounds
            area = cv2.contourArea(cntr)
            equi_diameter = np.sqrt(4*area/np.pi)
            
            x,y,obj_length,obj_height = cv2.boundingRect(cntr)

            equi_diameter = obj_length #bounding rectangle gives a better approximation of diameter
            
            #Hardcoded Diameter Range in pixels
            LOW_DIAMETER_BOUND = 50
            HIGH_DIAMETER_BOUND = 200
            
            HIGH_MM_BOUND = 400

            HIGH_DISTANCE_BOUND = 6000
            #Original tolerances were 20 and 150

            if(equi_diameter > LOW_DIAMETER_BOUND and equi_diameter < HIGH_DIAMETER_BOUND):
                mask = np.zeros_like(img) #mask will contain the fitted and adjusted ellipse of a single obstacle
                ellipse = cv2.fitEllipse(cntr)
                rect = cv2.minAreaRect(cntr)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                mask = cv2.ellipse(mask,ellipse,(255,255,255),-1) #draw the fitted ellipse
                rows = mask.shape[0]
                cols = mask.shape[1]
                M = np.float32([[1,0,0],[0,1,equi_diameter/4]]) #shift mask down to match obstacle, not edge
                mask = cv2.warpAffine(mask,M,(cols,rows))
                mask = cv2.erode(mask, kernel, iterations=3) #erode the mask to remove background points
                img_fg = cv2.bitwise_and(depth_frame,depth_frame,mask = mask) #use the mask to isolate original depth values
                img_fg = cv2.medianBlur(img_fg,5) #median blur to further remove noise
                obstacles = cv2.add(np.float32(img_fg), np.float32(obstacles)) 
                
                mean_val = np.median(img_fg[img_fg.nonzero()]) #compute the non-zero average of obstacle depth values
                min_val, distance_to_object, min_loc, max_loc = cv2.minMaxLoc(img_fg)
                mm_diameter = (equi_diameter) * (1.0 / CameraParams['fx']) * mean_val #convert pixel diameter to mm


                moment = cv2.moments(cntr) #get the centroid of the obstacle using its moment
                cx = int(moment['m10']/moment['m00'])
                cy = int(moment['m01']/moment['m00'])

                if mean_val < HIGH_DISTANCE_BOUND and mm_diameter < HIGH_MM_BOUND: #kinect loses accuracy beyond 4.5m
                    coords = depthToPointCloudPos(cx, cy, mean_val) #convert obstacle depth to XYZ coordinate
                    #begin visualization
                    cv2.drawContours(color,[box],0,(0,0,255),1)
                    cv2.rectangle(color,(x,y),(x+obj_length,y+obj_height),(0,255,0),2)
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    cv2.putText(color, "x" + str(coords[0]), (cx,cy+30), font, 0.4, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.putText(color, "y" + str(coords[1]), (cx,cy+45), font, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.putText(color, "z" + str(mean_val), (cx,cy+60), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(color,"diameter = " + str(mm_diameter), (cx,cy + 15), font, 0.4, (255, 0, 0), 1, cv2.LINE_AA)

        except:
            print ("Failed to process image")
            print (sys.exc_info()[0])
            
    elapsed_time = time.time() - start_time
    print("Frame took " + str(elapsed_time) + " seconds to process")

    cv2.imshow("plane",thresh)
    cv2.imshow("img",depth_frame/4500.0)
    cv2.imshow("final",color)
    cv2.imshow("denoised",denoised)
    cv2.imshow("obstacles",obstacles)
    key = cv2.waitKey(delay=1)
    if key == ord('q'):
        cv2.destroyAllWindows()
        sys.exit(0)
    return color



# Single Image Testing

In [12]:
img = np.load('test_frames_arc/429.npy')
output = get_obstacles_with_plane(img, num_planes = 10, 
                                  num_points = 2000, 
                                  dist_thresh = 0.5, 
                                  visualize = True)

[0.36645438 0.40350172 0.2464429 ] [ 1.25812116 -0.22018242 -1.03793874] -9.930815992141977
Frame took 0.34779930114746094 seconds to process


# Multiple Frame Testing

In [14]:
frames_dir = 'test_frames_arc/'
directory = os.fsencode(frames_dir)
frame_i = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".npy"):
        file_path = frames_dir + filename
        print(file_path)
        img = np.load(file_path)
        try:
            output = get_obstacles_with_plane(img, num_planes = 10, 
                                  num_points = 2000, 
                                  dist_thresh = 0.5, 
                                  visualize = False)
            cv2.imwrite(os.path.join(path ,filename[:-4] + '.png'), output)
            frame_i += 1
        except:
            print("Could not find obstacles")
cv2.destroyAllWindows()

C:\Users\jumpr_000\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  after removing the cwd from sys.path.


test_frames_arc/0.npy
[0.11810481 0.75361446 0.6328543 ] [ 1.25462601 -0.21819113 -1.03643488] -9.864228518369282
Frame took 0.49871277809143066 seconds to process
test_frames_arc/1.npy
[0.8282685  0.43643376 0.39255317] [ 1.25163864 -0.21108155 -1.04055709] -9.5757843011603
Frame took 0.45073962211608887 seconds to process
test_frames_arc/10.npy
[0.2218546  0.61880045 0.39766768] [ 1.25427644 -0.22116942 -1.03310702] -10.000327179686707
Frame took 0.46273374557495117 seconds to process
test_frames_arc/100.npy
[0.17999107 0.31027138 0.21555426] [ 1.26543473 -0.22040138 -1.04503334] -9.885591632881091
Frame took 0.35680580139160156 seconds to process
test_frames_arc/101.npy
[0.29819621 0.66148376 0.26221023] [ 1.26572928 -0.22158416 -1.04414512] -9.931272504524731
Frame took 0.3627967834472656 seconds to process
test_frames_arc/102.npy
[0.28111442 0.5199487  0.60915479] [ 1.2632875  -0.22665389 -1.03663362] -10.170984845108295
Frame took 0.3677840232849121 seconds to process
test_frames

[ 0.25136464  0.18510894 -0.01422305] [ 1.26332165 -0.22837633 -1.03494532] -10.252901798526272
Frame took 0.3277928829193115 seconds to process
test_frames_arc/144.npy
[0.44806803 0.38963285 0.94214773] [ 1.27007219 -0.230578   -1.03949419] -10.288140170421164
Frame took 0.34979844093322754 seconds to process
test_frames_arc/145.npy
[ 0.74709765 -0.18987803  0.45473123] [ 1.2630259  -0.22761958 -1.03540633] -10.216703378337957
Frame took 0.3237941265106201 seconds to process
test_frames_arc/146.npy
[ 0.02449124  0.1948568  -0.01382258] [ 1.26965271 -0.22740763 -1.04224507] -10.147091413677078
Frame took 0.3428025245666504 seconds to process
test_frames_arc/147.npy
[0.58349916 0.17391951 0.26757469] [ 1.26288084 -0.22700793 -1.03587291] -10.190632196495104
Frame took 0.35181641578674316 seconds to process
test_frames_arc/148.npy
[ 0.10086903 -0.19342158  0.53407584] [ 1.25493255 -0.22618792 -1.02874463] -10.219832756906225
Frame took 0.333789587020874 seconds to process
test_frames_arc

[0.45171047 0.08924624 0.26954749] [ 1.26881402 -0.23334853 -1.03546549] -10.419259299535963
Frame took 0.30780458450317383 seconds to process
test_frames_arc/19.npy
[0.41532478 0.30415404 0.35905452] [ 1.26196754 -0.22739946 -1.03456807] -10.21275514805554
Frame took 0.3727860450744629 seconds to process
test_frames_arc/190.npy
[0.30135413 0.26696649 0.42419317] [ 1.27543943 -0.24482151 -1.03061792] -10.866314821137113
Frame took 0.3168175220489502 seconds to process
test_frames_arc/191.npy
[0.02799402 0.08476083 0.16669613] [ 1.27033856 -0.23680788 -1.03353068] -10.562073992240006
Frame took 0.3088040351867676 seconds to process
test_frames_arc/192.npy
[0.28563416 0.375282   0.53977322] [ 1.28134536 -0.24470002 -1.03664534] -10.809597122924856
Frame took 0.30784034729003906 seconds to process
test_frames_arc/193.npy
[0.4975595  0.46326127 0.36806187] [ 1.2606683  -0.23983863 -1.02082967] -10.767981310295387
Frame took 0.31180238723754883 seconds to process
test_frames_arc/194.npy
[0.

[0.33256809 0.35377722 0.2608995 ] [ 1.26886031 -0.24494578 -1.02391454] -10.93090705118237
Frame took 0.3038201332092285 seconds to process
test_frames_arc/234.npy
[0.21781018 0.19785204 0.34010933] [ 1.28551325 -0.24146474 -1.04404851] -10.639673154815625
Frame took 0.29181861877441406 seconds to process
test_frames_arc/235.npy
[0.36499696 0.1963895  0.66559833] [ 1.29987369 -0.24916843 -1.05070526] -10.852518845259961
Frame took 0.28882837295532227 seconds to process
test_frames_arc/236.npy
[0.12035104 0.37692557 0.37310866] [ 1.27810461 -0.24566121 -1.03244339] -10.885929043595763
Frame took 0.2948477268218994 seconds to process
test_frames_arc/237.npy
[0.06076869 0.42060044 0.39105616] [ 1.2601931  -0.23362311 -1.02656999] -10.505984100970172
Frame took 0.3098015785217285 seconds to process
test_frames_arc/238.npy
[0.11847924 0.31070436 0.25185087] [ 1.26456878 -0.24378826 -1.02078052] -10.908206915639973
Frame took 0.29984593391418457 seconds to process
test_frames_arc/239.npy
[0

...

[0.66461648 0.32079044 0.36188515] [ 1.25629884 -0.24045276 -1.01584608] -10.842036114900063
Frame took 0.28381848335266113 seconds to process
test_frames_arc/35.npy
[ 0.09062116 -0.08359743  0.30397899] [ 1.25846098 -0.21403279 -1.04442819] -9.65257091735198
Frame took 0.38477373123168945 seconds to process
test_frames_arc/350.npy
[0.12739158 0.15475136 0.26659412] [ 1.26578034 -0.24179336 -1.02398698] -10.816202184392832
Frame took 0.2848351001739502 seconds to process
test_frames_arc/351.npy
[0.34132514 0.29497023 0.26825884] [ 1.2598249  -0.23540971 -1.02441519] -10.583280273964043
Frame took 0.30183911323547363 seconds to process
test_frames_arc/352.npy
[0.40311923 0.42689398 0.19330942] [ 1.27920558 -0.24112677 -1.03807881] -10.670689188055302
Frame took 0.27982354164123535 seconds to process
test_frames_arc/353.npy
[0.38147475 0.58206963 0.47642172] [ 1.2684942  -0.24342765 -1.02506655] -10.863606912949134
Frame took 0.2948305606842041 seconds to process
test_frames_arc/354.npy


[0.12746311 0.57624338 0.12394748] [ 1.2700088  -0.21305264 -1.05695616] -9.520789530608097
Frame took 0.3267972469329834 seconds to process
test_frames_arc/395.npy
[0.29074768 0.1430169  0.35569504] [ 1.25722761 -0.21588934 -1.04133827] -9.742197073663927
Frame took 0.32982301712036133 seconds to process
test_frames_arc/396.npy
[0.55374301 0.40449442 0.07365076] [ 1.27335364 -0.21762021 -1.05573343] -9.69898042408716
Frame took 0.32679271697998047 seconds to process
test_frames_arc/397.npy
[0.52338349 0.47456429 0.04405588] [ 1.25274621 -0.21037913 -1.04236708] -9.535113631505652
Frame took 0.3228147029876709 seconds to process
test_frames_arc/398.npy
[ 0.36103437 -0.10949054  0.0797021 ] [ 1.26146168 -0.21608289 -1.04537879] -9.720445323041421
Frame took 0.3248131275177002 seconds to process
test_frames_arc/399.npy
[0.15866358 0.73011402 0.1039864 ] [ 1.26073017 -0.21504621 -1.04568397] -9.674169201160774
Frame took 0.31679868698120117 seconds to process
test_frames_arc/4.npy
[0.2085

[ 0.21961405  0.07393777 -0.30940483] [ 1.25863051 -0.21321871 -1.0454118 ] -9.615091341203053
Frame took 0.31981563568115234 seconds to process
test_frames_arc/44.npy
[0.12684904 0.00891739 0.20353388] [ 1.26431373 -0.21973556 -1.04457817] -9.857668452419471
Frame took 0.3997642993927002 seconds to process
test_frames_arc/440.npy
[0.23706586 0.47496124 0.55458386] [ 1.25924075 -0.21527452 -1.04396624] -9.705201230125272
Frame took 0.33478879928588867 seconds to process
test_frames_arc/441.npy
[0.36066351 0.36699135 0.1434345 ] [ 1.25341372 -0.21620972 -1.03720401] -9.788831893865098
Frame took 0.3268122673034668 seconds to process
test_frames_arc/442.npy
[0.31170593 0.39295362 0.08220705] [ 1.2539364  -0.21036693 -1.04356948] -9.529434769537868
Frame took 0.31879758834838867 seconds to process
test_frames_arc/443.npy
[0.49394438 0.48518286 0.52940764] [ 1.25496892 -0.20949296 -1.04547596] -9.470703550626554
Frame took 0.33981823921203613 seconds to process
test_frames_arc/444.npy
[0.2

[0.13038526 0.23655533 0.08366726] [ 1.28681619 -0.23263525 -1.05418094] -10.249172769478038
Frame took 0.30182647705078125 seconds to process
test_frames_arc/485.npy
[ 0.13712907 -0.07135496  0.36473758] [ 1.27100757 -0.23143167 -1.0395759 ] -10.317096922486073
Frame took 0.29381299018859863 seconds to process
test_frames_arc/486.npy
[0.23750323 0.24124331 0.10863948] [ 1.27609144 -0.23211198 -1.04397946] -10.309704400454851
Frame took 0.302807092666626 seconds to process
test_frames_arc/487.npy
[0.43503329 0.27004761 0.31971104] [ 1.27827856 -0.23576591 -1.04251265] -10.445743853331024
Frame took 0.3138096332550049 seconds to process
test_frames_arc/488.npy
[0.41091933 0.60033084 0.28806199] [ 1.26235667 -0.22708157 -1.03527511] -10.19899901027721
Frame took 0.3048062324523926 seconds to process
test_frames_arc/489.npy
[0.76503141 0.18230786 0.58358968] [ 1.2626101  -0.22665082 -1.03595928] -10.17092962726655
Frame took 0.30182623863220215 seconds to process
test_frames_arc/49.npy
[ 

[0.57139124 0.20509563 0.73260635] [ 1.26863503 -0.23215703 -1.03647799] -10.370277265717284
Frame took 0.3947865962982178 seconds to process
test_frames_arc/80.npy
[0.44271681 0.23103483 0.35284244] [ 1.27848269 -0.22264589 -1.0558368 ] -9.878248895536268
Frame took 0.3547959327697754 seconds to process
test_frames_arc/81.npy
[ 0.19115107  0.62006532 -0.12826774] [ 1.28145344 -0.22473085 -1.05672259] -9.949675604480834
Frame took 0.34580135345458984 seconds to process
test_frames_arc/82.npy
[0.05140883 0.58257608 0.03814913] [ 1.25781752 -0.21796888 -1.03984865] -9.827603051260159
Frame took 0.3467998504638672 seconds to process
test_frames_arc/83.npy
[0.45511112 0.41863807 0.29389865] [ 1.25635895 -0.21626417 -1.04009478] -9.768569850277544
Frame took 0.35079193115234375 seconds to process
test_frames_arc/84.npy
[0.16281144 0.20359804 0.28510989] [ 1.27944779 -0.21728516 -1.06216263] -9.640411539663557
Frame took 0.3527958393096924 seconds to process
test_frames_arc/85.npy
[0.4692595